

#  1 经典图神经网络（GNN）

图神经网络（Graph Neural Network）是一类**直接在图结构上定义的深度学习方法**，通过迭代地传播和聚合邻居信息来学习节点的表示。

---

## 1.1 神经消息传递机制（Neural Message Passing）

---

###  核心思想

神经消息传递（Message Passing）框架是**现代 GNN 的统一形式**，主要由两步组成：

1. **消息聚合（Aggregate）：**

   * 从邻居节点收集信息

2. **状态更新（Update）：**

   * 融合聚合后的邻居信息与当前节点自身状态

---

###  数学表达（第 $k$ 层）：

设节点 $v$ 的表示为 $h_v^{(k)}$，邻居为 $\mathcal{N}(v)$，有：

$$
m_v^{(k)} = \text{AGGREGATE}^{(k)}\left( \{ h_u^{(k-1)} \mid u \in \mathcal{N}(v) \} \right)
$$

$$
h_v^{(k)} = \text{UPDATE}^{(k)}\left( h_v^{(k-1)}, m_v^{(k)} \right)
$$

---

###  示例聚合函数

* 平均（Mean）
* 加权和（Sum / Attention）
* 最大池化（Max-Pooling）

---

###  核心问题

* 聚合函数是否可微？
* 聚合方式是否具备**判别力**？
* 是否能够区分图结构不同的节点？

>  GIN 被证明具有最大判别力；GCN 更偏线性低通滤波器；GAT 提供选择性聚合能力。

---

## 1.2 GraphSAGE（Inductive GraphSAGE）

---

> **Graph Sample and Aggregation (SAGE)**
> —— Hamilton et al., NeurIPS 2017

---

### 设计动机

早期方法如 GCN 假设全图可见，**仅支持 Transductive 学习（传导学习）**，不支持新节点或新图的泛化。

GraphSAGE 提出：

* 支持 **Inductive Learning**（归纳学习）
* 利用**可学习的聚合函数**对邻居表示进行采样和汇总

---

###  模型结构

第 $k$ 层表示更新为：

$$
h_v^{(k)} = \sigma \left( W^{(k)} \cdot \text{CONCAT} \left( h_v^{(k-1)}, \text{AGG}_{k} \left( \{ h_u^{(k-1)} : u \in \mathcal{N}(v) \} \right) \right) \right)
$$

---

###  常见的聚合器 AGG：

| 聚合器类型             | 描述                 |
| ----------------- | ------------------ |
| MeanAggregator    | 对邻居取均值             |
| PoolingAggregator | 对邻居用 MLP 投影后取 max  |
| LSTMAggregator    | 将邻居输入 LSTM         |
| GCNAggregator     | 邻接归一化后加权求和（类似 GCN） |

---

### 关键特性

* **局部采样聚合**，适合处理大图（避免全图传播开销）
* **归纳能力强**：能处理训练集中未出现的节点或子图
* 易扩展：支持属性、多图、多关系等场景

---

## 1.3 GAT（Graph Attention Network）

---

> **Graph Attention Networks**
> —— Veličković et al., ICLR 2018

---

### 核心思想

在传统 GNN 中，所有邻居信息聚合权重**固定或均等**。
GAT 引入**注意力机制（self-attention）**，自动学习邻居节点对目标节点的重要性。

---

### 结构流程（单头注意力）：

1. **线性变换：**

   $$
   h_i' = W h_i
   $$

2. **计算注意力权重（打分）：**

   $$
   e_{ij} = \text{LeakyReLU}\left( \vec{a}^\top [h_i' \| h_j'] \right)
   $$

3. **归一化（Softmax）：**

   $$
   \alpha_{ij} = \frac{ \exp(e_{ij}) }{ \sum_{k \in \mathcal{N}(i)} \exp(e_{ik}) }
   $$

4. **聚合邻居：**

   $$
   h_i^{(k)} = \sigma\left( \sum_{j \in \mathcal{N}(i)} \alpha_{ij} h_j' \right)
   $$

---

### 多头注意力（Multi-head Attention）

在每一层使用 $K$ 个独立注意力头并聚合：

$$
h_i^{(k)} = \bigg\|_{k=1}^{K} \sigma\left( \sum_{j \in \mathcal{N}(i)} \alpha_{ij}^{(k)} W^{(k)} h_j \right)
$$

---

### 特点与优势

| 特性      | 优势             |
| ------- | -------------- |
| 可学习权重   | 区分重要邻居与无关邻居    |
| 无需归一化矩阵 | 适合异构图、稀疏图      |
| 结构可解释   | 注意力得分可视化分析模型行为 |
| 多头增强稳定性 | 多视角聚合，有助于泛化能力  |

---

## 小结：经典 GNN 模型对比

| 模型        | 聚合方式   | 是否归纳 | 是否加权 | 特点        |
| --------- | ------ | ---- | ---- | --------- |
| GCN       | 邻接归一化  | ✖️   | ✖️   | 层间权重共享，简洁 |
| GraphSAGE | 可学习聚合器 | ✅    | 可选   | 可扩展性好     |
| GAT       | 注意力机制  | ✅    | ✅    | 可解释性强     |







#  2 卷积（Convolution）

---

## 2.1 数学上的卷积（Convolution in Mathematics）

---

###  定义（连续情形）

在数学分析中，**卷积**是两个函数的一种积分变换形式，用于描述一个函数如何“影响”另一个函数：

$$
(f * g)(t) = \int_{-\infty}^{+\infty} f(\tau) \cdot g(t - \tau) \, d\tau
$$

* $f$：输入函数
* $g$：核函数（如滤波器）
* $t$：当前位置
* $*$：卷积操作符

---

###  离散卷积

$$
(f * g)[n] = \sum_{m = -\infty}^{+\infty} f[m] \cdot g[n - m]
$$

* 典型于数字信号处理中使用

---

###  特点

* 平移不变性（Shift-Invariance）
* 局部性（Locality）
* 可用作滤波器（平滑、边缘检测等）

---

###  例子

* 模糊图像：用高斯核与图像做卷积
* 边缘检测：用 Sobel 核对图像卷积

---

## 2.2 计算机视觉上的卷积（Convolution in Computer Vision）

---

### 图像卷积（Image Convolution）

图像是二维离散信号，卷积操作常用来提取图像的局部特征：

$$
Y[i, j] = \sum_{m} \sum_{n} X[i + m, j + n] \cdot K[m, n]
$$

* $X$：输入图像
* $K$：卷积核（通常为 $3 \times 3$, $5 \times 5$...）
* $Y$：输出特征图

---

### 卷积神经网络（CNN）结构：

1. **卷积层（Convolution Layer）**：提取局部特征
2. **激活层（ReLU）**：引入非线性
3. **池化层（Pooling）**：降维，增强平移不变性
4. **全连接层（FC）**：最终决策或分类

---

###  CNN 卷积的关键特性

| 特性    | 含义           |
| ----- | ------------ |
| 权重共享  | 同一个卷积核作用于整张图 |
| 局部连接  | 卷积核只作用于局部区域  |
| 平移不变性 | 滤波器在空间移动仍有效  |

---

###  问题引出：**图结构能否使用卷积？**

> 图结构没有固定的“邻接规则”或“网格结构”，传统 CNN 不适用于图。

---

## 2.3 从图像卷积到图卷积（Graph Convolution）

![](./img/3_1.png)
![](./img/3_2.png)
![](./img/3_3.png)
![](./img/3_4.png)

---

### 挑战：图结构的非欧几里得特性

| 图像结构    | 图结构       |
| ------- | --------- |
| 规则网格    | 不规则连通结构   |
| 固定邻接方式  | 可变邻居数量    |
| 卷积核滑动   | 无“滑动窗口”定义 |
| 邻居位置有顺序 | 邻居无顺序     |

---

### 图卷积的目标

在图上定义一种“类似卷积”的操作，使得：

* 每个节点能接收来自邻居的信息
* 模型可学习**局部图结构特征**

---

### 图卷积的两大流派：

| 方法                 | 说明                  | 代表            |
| ------------------ | ------------------- | ------------- |
| **谱域方法（Spectral）** | 利用图傅里叶变换，将卷积定义为频域乘积 | GCN、ChebNet   |
| **空间方法（Spatial）**  | 在节点的邻居中直接进行聚合操作     | GraphSAGE、GAT |

---

### 1）谱域图卷积：从频域角度定义卷积

参考信号处理中的理论，利用图拉普拉斯矩阵 $L$ 做图傅里叶变换：

* 图信号 $x \in \mathbb{R}^n$
* 拉普拉斯特征分解：$L = U \Lambda U^\top$
* 图卷积定义为：

$$
g_\theta * x = U g_\theta(\Lambda) U^\top x
$$

* $g_\theta(\Lambda)$：频域滤波器函数
* $U$：图傅里叶基
* 缺点：代价高、不能泛化到不同图

---

### 2）GCN（简化谱域模型）

Kipf & Welling（2017）对谱方法近似处理，提出了 GCN 模型：

$$
H^{(l+1)} = \sigma \left( \tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2} H^{(l)} W^{(l)} \right)
$$

* $\tilde{A} = A + I$：加入自连接
* $\tilde{D}$：对应度矩阵
* $H^{(l)}$：第 $l$ 层节点表示
* 核心思想是**邻居特征的归一化加权聚合**

---

### 3）空间域图卷积：邻居聚合范式

空间方法不涉及频域变换，直接使用消息传递方式：

$$
h_v^{(k)} = \text{UPDATE} \left( h_v^{(k-1)}, \text{AGG} \left( \{ h_u^{(k-1)} \mid u \in \mathcal{N}(v) \} \right) \right)
$$

代表模型：

* GraphSAGE
* GAT
* GIN

---

### 图像 vs 图结构 卷积对比

| 属性   | 图像卷积        | 图结构卷积       |
| ---- | ----------- | ----------- |
| 数据结构 | 规则网格        | 任意图结构       |
| 邻域数量 | 固定          | 不定          |
| 邻居顺序 | 有           | 无           |
| 卷积定义 | 滑动窗口 + 权重共享 | 邻居聚合 + 权重学习 |
| 实现方式 | 空间域滑动       | 谱域 or 空间聚合  |

---

## 小结

| 概念   | 核心思想         | 代表方法                  |
| ---- | ------------ | --------------------- |
| 数学卷积 | 滤波器在函数上的滑动与积 | 傅里叶分析                 |
| 图像卷积 | 卷积核提取局部图像特征  | CNN                   |
| 图卷积  | 利用邻居聚合生成节点嵌入 | GCN / GAT / GraphSAGE |





#  3 图傅立叶变换

---

## 3.1 卷积与傅立叶变换

---

###  卷积定理（Convolution Theorem）

在经典信号处理领域中，**卷积和傅立叶变换之间有一个核心定理**：

> **时域的卷积等价于频域的乘积。**

$$
f * g = \mathcal{F}^{-1}( \mathcal{F}(f) \cdot \mathcal{F}(g) )
$$

其中：

* $f, g$：两个信号
* $\mathcal{F}$：傅立叶变换
* $\mathcal{F}^{-1}$：傅立叶逆变换

---

###  反之亦然：

> 频域的卷积等价于时域的乘积。

这一定理为我们在图上定义“卷积”提供了启发：
→ **如果我们能在图上构造傅立叶变换，就能在图上定义卷积操作。**

---

## 3.2 傅立叶变换回顾（Fourier Transform）

---

###  连续傅立叶变换

将函数 $f(t)$ 分解为不同频率的正弦/余弦波之和：

$$
\hat{f}(\omega) = \int_{-\infty}^{+\infty} f(t) \cdot e^{-i\omega t} \, dt
$$

* $\omega$：频率变量
* $\hat{f}(\omega)$：频域表示

---

###  离散傅立叶变换（DFT）

对于离散信号 $f \in \mathbb{R}^n$，其 DFT 表示为：

$$
\hat{f}[k] = \sum_{n=0}^{N-1} f[n] \cdot e^{-2\pi i k n / N}
$$

可写成矩阵形式：

$$
\hat{f} = F f
\quad \text{其中 } F \text{ 是傅立叶变换矩阵}
$$

---

###  应用直觉

* 低频分量表示信号的全局趋势
* 高频分量表示信号的局部变化

→ **滤波器（如低通、高通）可通过频域乘法实现特征提取**

---

## 3.3 图卷积的定义

---

传统 CNN 的卷积依赖于规则网格结构（如图像），但图是**非欧结构（non-Euclidean）**。
因此需借助图信号处理的工具，在图上定义卷积操作。

---

###  核心想法

> 用 **图傅立叶变换** 替代欧几里得傅立叶变换，在频域定义卷积操作。

---

## 3.4 图傅立叶变换（Graph Fourier Transform）

---

###  图信号定义

设图 $G = (V, E)$，有 $n$ 个节点，图信号为：

$$
x \in \mathbb{R}^n, \quad \text{每个元素 } x_i \text{ 是节点 } v_i \text{ 的特征}
$$

---

###  拉普拉斯矩阵（Graph Laplacian）

定义无向图的拉普拉斯算子（见下文第 3.5 节）：

$$
L = D - A
\quad \text{或归一化形式} \quad L_{sym} = I - D^{-1/2} A D^{-1/2}
$$

* $A$：邻接矩阵
* $D$：度矩阵（对角矩阵）

---

###  谱分解（Spectral Decomposition）

L 是对称实矩阵，可特征分解为：

$$
L = U \Lambda U^\top
$$

* $U \in \mathbb{R}^{n \times n}$：特征向量矩阵（正交基）
* $\Lambda$：特征值对角矩阵

---

###  图傅立叶变换定义

* **正变换（频域）**：

$$
\hat{x} = U^\top x
$$

* **逆变换（回到节点域）**：

$$
x = U \hat{x}
$$

即图信号 $x$ 的傅立叶变换就是**投影到 Laplacian 的特征向量基上**

---

###  图卷积定义（频域）：

定义图卷积为：

$$
g_\theta * x = U g_\theta(\Lambda) U^\top x
$$

其中：

* $g_\theta(\Lambda)$：可学习的滤波函数（如多项式或对角权重）
* $U^\top x$：信号的图傅立叶变换
* $U g_\theta(\Lambda) U^\top$：整个滤波器

---

###  解释：

* 与经典傅立叶类似，这里是**频域乘法**
* $g_\theta(\Lambda)$：频率滤波器，控制保留/抑制哪些谱分量
* 核心计算开销在于 $U$（需做特征分解）

---

## 3.5 图拉普拉斯（Graph Laplacian）

---

图拉普拉斯算子是图信号处理中最重要的结构工具。

---

###  非归一化拉普拉斯

$$
L = D - A
$$

* $D$：节点度矩阵
* $A$：邻接矩阵

---

###  对称归一化拉普拉斯（Symmetric Normalized Laplacian）

$$
L_{sym} = I - D^{-1/2} A D^{-1/2}
$$

---

###  随机归一化拉普拉斯（Random Walk Laplacian）

$$
L_{rw} = I - D^{-1} A
$$

---

###  性质总结

| 类型        | 矩阵表达                      | 是否对称 | 是否归一化 | 是否正定 |
| --------- | ------------------------- | ---- | ----- | ---- |
| $L$       | $D - A$                   | ✅    | ✖️    | 半正定  |
| $L_{sym}$ | $I - D^{-1/2} A D^{-1/2}$ | ✅    | ✅     | 半正定  |
| $L_{rw}$  | $I - D^{-1} A$            | ✖️   | ✅     | 半正定  |

---

###  图傅立叶基

* 拉普拉斯矩阵的特征向量组成正交基 $U$
* 特征值反映频率大小（越大越高频）
* 类似 DFT 中的正弦波基函数

---

###  Chebyshev 多项式近似（ChebNet）

为避免计算 $U$，可使用 Chebyshev 多项式近似滤波器：

$$
g_\theta * x \approx \sum_{k=0}^K \theta_k T_k(\tilde{L}) x
\quad \text{其中 } \tilde{L} = \frac{2L}{\lambda_{\max}} - I
$$

* $T_k$：第 $k$ 阶 Chebyshev 多项式
* 快速计算、局部化特性强

---

##  小结

| 概念     | 含义                   |
| ------ | -------------------- |
| 图傅立叶变换 | 将图信号映射到频域（图拉普拉斯特征空间） |
| 图卷积    | 在频域对信号进行滤波再映射回节点域    |
| 拉普拉斯算子 | 描述图结构差异性，提供谱分解基      |
| GCN    | 用拉普拉斯构造近似卷积核，实现局部聚合  |





#  4 卷积图神经网络实例

谱域图卷积网络以图傅里叶变换为基础，通过在**频域对图信号进行滤波**，实现图卷积操作。其核心优势在于：

* 理论完备（基于图信号处理）
* 易于分析频率响应
* 可解释性强

---

## 4.1 Spectral GCN（频谱卷积网络）

---

> 由 Bruna et al. 在 2013 年首次提出，将卷积定义在图的频谱空间。

---

###  模型定义

设图拉普拉斯矩阵 $L = U \Lambda U^\top$，其特征分解提供傅里叶基：

图卷积操作定义为：

$$
g_\theta * x = U \cdot g_\theta(\Lambda) \cdot U^\top x
$$

* $x \in \mathbb{R}^n$：图信号（节点特征）
* $g_\theta(\Lambda)$：频域滤波器（对角矩阵）
* $U$：图傅里叶基（L 的特征向量）

---

###  缺点

1. **无法泛化到其他图**
   特征基 $U$ 依赖图结构，换图需重新计算特征分解。

2. **计算复杂度高**
   $O(n^3)$ 的特征分解，难以扩展到大规模图。

3. **滤波器非局部**
   导致特征跨越整个图，无法保证局部性。

---

## 4.2 ChebyNet（Chebyshev Spectral GCN）

---

> 提出者：Defferrard et al., NIPS 2016
> 目标：解决 Spectral GCN 的三个缺陷

---

###  核心思想：用**Chebyshev 多项式**近似滤波函数

将频域滤波器 $g_\theta(\Lambda)$ 近似成 $K$ 阶 Chebyshev 多项式：

$$
g_\theta(\Lambda) \approx \sum_{k=0}^{K} \theta_k T_k(\tilde{\Lambda})
\quad \Rightarrow \quad
g_\theta * x \approx \sum_{k=0}^{K} \theta_k T_k(\tilde{L}) x
$$

* $T_k(\cdot)$：第 $k$ 阶 Chebyshev 多项式，递归定义如下：

  $$
  T_0(x) = 1, \quad T_1(x) = x, \quad T_{k}(x) = 2xT_{k-1}(x) - T_{k-2}(x)
  $$

* $\tilde{L} = \frac{2L}{\lambda_{\max}} - I$：缩放后的拉普拉斯，谱范围在 $[-1,1]$

---

###  模型结构

图卷积操作：

$$
g_\theta * x = \sum_{k=0}^K \theta_k T_k(\tilde{L}) x
$$

* **无需特征分解！**
* **具有 $K$-阶邻域的局部性**
* 参数量随 $K$ 线性增长

---

###  特点总结

| 特点      | 描述                |
| ------- | ----------------- |
| 局部性     | 支持 $K$-hop 局部信息传播 |
| 高效      | 避免特征分解            |
| 泛化性     | 适用于任意图            |
| 近似表达能力强 | 通过多项式构建复杂滤波器      |

---

### 小结图示

```
原始频谱卷积 → 用切比雪夫多项式拟合 → 空间中实现快速局部传播
```

---

## 4.3 ChebyNet-GCN（GCN 是 ChebyNet 的特例）

---

> Kipf & Welling 在 ICLR 2017 提出的 GCN 可以看作是 ChebNet 的**一阶近似版本**。

---

###  从 ChebNet 到 GCN

取 $K = 1$，并对切比雪夫展开进行简化：

$$
g_\theta * x \approx \theta_0 x + \theta_1 \tilde{L} x
\quad \text{其中 } \tilde{L} = \frac{2L}{\lambda_{\max}} - I
$$

设 $\theta = \theta_0 = -\theta_1$，得到：

$$
g_\theta * x \approx \theta (I + D^{-1/2} A D^{-1/2}) x
$$

再加入非线性变换和参数学习权重，构建 GCN 层：

$$
H^{(l+1)} = \sigma \left( \tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2} H^{(l)} W^{(l)} \right)
$$

* $\tilde{A} = A + I$：添加自环
* $\tilde{D}$：对应的度矩阵
* $H^{(l)}$：第 $l$ 层特征矩阵

---

### 核心优点

| 特性   | 描述          |
| ---- | ----------- |
| 局部传递 | 聚合一阶邻居特征    |
| 参数共享 | 所有节点使用相同权重  |
| 高效   | 矩阵乘法即可实现图卷积 |

---

### 缺点

* 层数过多会导致“过平滑”（oversmoothing）问题
* 固定邻居聚合权重，表达能力有限

---

## 三者比较小结

| 模型           | 核心思想           | 参数       | 是否近似 | 局部性 | 计算复杂度    |
| ------------ | -------------- | -------- | ---- | --- | -------- |
| Spectral GCN | 精确傅立叶变换        | $n$ 个特征值 | 否    | 否   | $O(n^3)$ |
| ChebyNet     | 多项式近似谱滤波       | $K$ 个    | 是    | ✅   | 线性复杂度    |
| GCN          | ChebyNet 的一阶特例 | 少量参数     | 是    | ✅   | 极高效率     |

